# Experiment Builder — Examples

This notebook shows how to build experiment *plans* (CSV grids) decoupled from actually running generation.

You can inspect the planned runs (Cartesian product) before spending compute.


In [1]:


from experiments.build.experiment_builder import ExperimentSpec, build_and_write
from guidance_registry import SCHEDULER_PROPERTIES

list(SCHEDULER_PROPERTIES.keys())


['linear',
 'cosine',
 'exponential',
 'logarithmic',
 'sigmoid',
 'triangular',
 'parabolic',
 'constant',
 'baseline',
 'average_constant']

## Example A — Core sweep (baseline)

- Compare multiple scheduler kinds × directions × prompts with a standard guidance range.
- Intended for broad coverage.


In [8]:
EXPERIMENT = "EXP_A_CORE_SWEEP"

kinds = list(SCHEDULER_PROPERTIES.keys())
directions = ["increasing", "decreasing"]
weight_ranges = [(1.0, 7.5)]
seeds = [42, 1337, 2000]
num_steps = [25]

prompts = {
    "P_DETAIL": "A close-up portrait of a cyberpunk robot with intricate clockwork gears and neon eyes, macro photography",
    "P_COMP":   "A symmetrical wide shot of a lone tree in a snowy field, Wes Anderson style, pastel colors",
    "P_TEXT":   "A wooden sign hanging on a door that says 'CLOSED' clearly carved into the wood"
}

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "baseline sweep", "model": "SD1.5"},
    num_workers=3
)

df_a = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_a

,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,model,worker_id
0,EXP_A_CORE_SWEEP,50831c599181,linear,increasing,1.0,7.5,25,42,P_DETAIL,A close-up portrait of a cyberpunk robot with ...,{},baseline sweep,SD1.5,0
1,EXP_A_CORE_SWEEP,03b11ac95b8b,linear,increasing,1.0,7.5,25,43,P_DETAIL,A close-up portrait of a cyberpunk robot with ...,{},baseline sweep,SD1.5,1
2,EXP_A_CORE_SWEEP,05d61aad41da,linear,increasing,1.0,7.5,25,42,P_COMP,A symmetrical wide shot of a lone tree in a sn...,{},baseline sweep,SD1.5,2
3,EXP_A_CORE_SWEEP,f7cab3859543,linear,increasing,1.0,7.5,25,43,P_COMP,A symmetrical wide shot of a lone tree in a sn...,{},baseline sweep,SD1.5,0
4,EXP_A_CORE_SWEEP,51af3ffbf734,linear,increasing,1.0,7.5,25,42,P_TEXT,A wooden sign hanging on a door that says 'CLO...,{},baseline sweep,SD1.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,EXP_A_CORE_SWEEP,86124dc3e3dd,constant,increasing,1.0,7.5,25,43,P_DETAIL,A close-up portrait of a cyberpunk robot with ...,{},baseline sweep,SD1.5,1
86,EXP_A_CORE_SWEEP,7c59c2f5856a,constant,increasing,1.0,7.5,25,42,P_COMP,A symmetrical wide shot of a lone tree in a sn...,{},baseline sweep,SD1.5,2
87,EXP_A_CORE_SWEEP,7153f3c9da54,constant,increasing,1.0,7.5,25,43,P_COMP,A symmetrical wide shot of a lone tree in a sn...,{},baseline sweep,SD1.5,0
88,EXP_A_CORE_SWEEP,c518073a58b2,constant,increasing,1.0,7.5,25,42,P_TEXT,A wooden sign hanging on a door that says 'CLO...,{},baseline sweep,SD1.5,1


## Example B — Make the alignment ↔ fidelity tradeoff obvious (extreme ranges + “checkable” prompts)

Idea:
- Use **very low** and **very high** guidance ranges.
- Use prompts with *verifiable* details (counting, text, specific attributes).
- Goal is to clearly demonstrate failure modes: too-low guidance ignores prompt, too-high guidance overconstrains / artifacts.


In [10]:
EXPERIMENT = "EXP_B_EXTREME_TRADEOFF"

kinds = ["constant", "linear", "cosine", "sigmoid", "exponential"]
directions = ["increasing", "decreasing"]
weight_ranges = [
    (1.0, 2.0),   # very weak guidance band
    (1.0, 12.0),  # very strong ceiling (often causes collapse/artifacts)
]
seeds = [0, 1, 2, 3]
num_steps = [25]

prompts = {
    "P_COUNT": "A photo of exactly THREE red apples on a white plate, shot from above, sharp focus",
    "P_TEXT":  "A storefront window sign that clearly reads 'OPEN 24 HOURS' in bold black letters",
    "P_ATTR":  "A blue sports car with a yellow racing stripe, parked on wet asphalt at night, reflections visible",
}

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


(           experiment_group experiment_id      kind   direction  w_min  w_max  \
 151  EXP_B_EXTREME_TRADEOFF  0142f920bbbe   sigmoid  decreasing    1.0    2.0   
 56   EXP_B_EXTREME_TRADEOFF  36bbbc1c4a2a    linear  decreasing    1.0    2.0   
 123  EXP_B_EXTREME_TRADEOFF  3a7cd8406c58   sigmoid  increasing    1.0    2.0   
 60   EXP_B_EXTREME_TRADEOFF  f7dd9d2e8c0c    linear  decreasing    1.0   12.0   
 100  EXP_B_EXTREME_TRADEOFF  9ede8a02bd6a    cosine  decreasing    1.0    2.0   
 51   EXP_B_EXTREME_TRADEOFF  9df665471bb7    linear  decreasing    1.0    2.0   
 7    EXP_B_EXTREME_TRADEOFF  267286caca94  constant  increasing    1.0    2.0   
 5    EXP_B_EXTREME_TRADEOFF  ce0e6b9fc523  constant  increasing    1.0    2.0   
 
      num_steps  seed prompt_id  \
 151         25     3    P_TEXT   
 56          25     0    P_ATTR   
 123         25     3   P_COUNT   
 60          25     0   P_COUNT   
 100         25     0    P_TEXT   
 51          25     3   P_COUNT   
 7           25

In [7]:
EXPERIMENT = "EXP_B_EXTREME_TRADEOFF_R2"

kinds = ["baseline", "average_constant", "constant", "linear", "exponential", "triangular"]
directions = ["increasing", "decreasing"]
weight_ranges = [(1.0, 7.5), (7.5, 20), (7.5, 30), (7.5, 40), (7.5, 50), (7.5, 60), (7.5, 70), (7.5, 90)]

seeds = [1337]
num_steps = [40]

prompts = {
    "P_APPLE_COUNT_DETAILED": "masterpiece, best quality, 3 red apples on a wooden table, studio lighting, sharp focus",
    }

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details", "sampler" : "Euler"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,bucket,sampler,worker_id
0,EXP_B_EXTREME_TRADEOFF_R2,cecc162be647,baseline,increasing,1.0,7.5,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
1,EXP_B_EXTREME_TRADEOFF_R2,cecc162be647,baseline,increasing,7.5,20.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,1
2,EXP_B_EXTREME_TRADEOFF_R2,cecc162be647,baseline,increasing,7.5,30.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,2
3,EXP_B_EXTREME_TRADEOFF_R2,cecc162be647,baseline,increasing,7.5,40.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,3
4,EXP_B_EXTREME_TRADEOFF_R2,cecc162be647,baseline,increasing,7.5,50.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,EXP_B_EXTREME_TRADEOFF_R2,50a9424d8924,triangular,decreasing,7.5,40.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,3
68,EXP_B_EXTREME_TRADEOFF_R2,05271c2f20ad,triangular,decreasing,7.5,50.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
69,EXP_B_EXTREME_TRADEOFF_R2,9fb4aaaa7c2d,triangular,decreasing,7.5,60.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,1
70,EXP_B_EXTREME_TRADEOFF_R2,b41e4a797dd2,triangular,decreasing,7.5,70.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,2


In [8]:
EXPERIMENT = "EXP_B_EXTREME_TRADEOFF_100"

kinds = ["constant", "linear", "exponential", "triangular"]
directions = ["increasing", "decreasing"]
weight_ranges = [(7.5, 100)]

seeds = [1337]
num_steps = [40]

prompts = {
    "P_APPLE_COUNT_DETAILED": "masterpiece, best quality, 3 red apples on a wooden table, studio lighting, sharp focus",
    }

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details", "sampler" : "Euler"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,bucket,sampler,worker_id
0,EXP_B_EXTREME_TRADEOFF_100,c952ee58d52d,constant,increasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
1,EXP_B_EXTREME_TRADEOFF_100,87139bf0707b,linear,increasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,1
2,EXP_B_EXTREME_TRADEOFF_100,02e5d7709558,linear,decreasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,2
3,EXP_B_EXTREME_TRADEOFF_100,200bbe8d2b28,exponential,increasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,3
4,EXP_B_EXTREME_TRADEOFF_100,987784264bce,exponential,decreasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
5,EXP_B_EXTREME_TRADEOFF_100,40c50437f7e6,triangular,increasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,1
6,EXP_B_EXTREME_TRADEOFF_100,9ed85a63d6f3,triangular,decreasing,7.5,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,2


In [2]:
EXPERIMENT = "EXP_B_for_abstract_apple_results"

kinds = ["baseline", "constant", "linear"]
directions = ["decreasing"]
weight_ranges = [(7.0, 50), (7.0, 100)]

seeds = [1337]
num_steps = [40]

prompts = {
    "P_APPLE_COUNT_DETAILED": "masterpiece, best quality, 3 red apples on a wooden table, studio lighting, sharp focus",
    }

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details", "sampler" : "Euler"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,bucket,sampler,worker_id
0,EXP_B_for_abstract_apple_results,0573ef25ccc7,constant,increasing,7.0,50.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
1,EXP_B_for_abstract_apple_results,bceada482242,constant,increasing,7.0,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,1
2,EXP_B_for_abstract_apple_results,2bbca14fabec,linear,increasing,7.0,50.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,2
3,EXP_B_for_abstract_apple_results,28f667f93ee0,linear,increasing,7.0,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,3
4,EXP_B_for_abstract_apple_results,5f631e2f2ca7,linear,decreasing,7.0,50.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,0
5,EXP_B_for_abstract_apple_results,522c94e6c302,linear,decreasing,7.0,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,Euler,1


## Colors Experiment
## Example C — “Out-of-distribution” steering hypothesis (increasing guidance)


Hypothesis:
- For prompts that are *hard / out-of-distribution*, starting with lower guidance may keep samples on-manifold early,
  then increasing guidance later might steer fine details toward adherence.

We encode that by focusing on **increasing** schedules and optionally stronger ceilings.


In [3]:
EXPERIMENT = "EXP_C_EXTREME_TRADEOFF_COLORS"

kinds = ["baseline", "average_constant", "constant", "linear", "exponential", "triangular"]
directions = ["increasing", "decreasing"]
weight_ranges = [
    (1.0, 2.0),   # very weak guidance band
    (1.0, 12.0),  # very strong ceiling (often causes collapse/artifacts)
    (7,50),
    (7,100),
]
seeds = [42, 1337]
num_steps = [40]

prompts = {
    "P_MUG_BOOK_COLORS": "A solid yellow coffee mug resting on a bright blue book.",
    "P_HAT_JACKET_COLORS" : "A man wearing a bright purple beanie and a neon orange winter jacket.",
    "P_CAR_HYDRANT_COLORS" : "A green sports car parked next to a red fire hydrant."
}

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,bucket,worker_id
0,EXP_C_EXTREME_TRADEOFF_COLORS,4c6be3aad94e,baseline,increasing,1.0,2.0,40,42,P_MUG_BOOK_COLORS,A solid yellow coffee mug resting on a bright ...,{},tradeoff demo,checkable-details,0
1,EXP_C_EXTREME_TRADEOFF_COLORS,34391b7b795b,baseline,increasing,1.0,2.0,40,1337,P_MUG_BOOK_COLORS,A solid yellow coffee mug resting on a bright ...,{},tradeoff demo,checkable-details,1
2,EXP_C_EXTREME_TRADEOFF_COLORS,5ca411ca4ab4,baseline,increasing,1.0,2.0,40,42,P_HAT_JACKET_COLORS,A man wearing a bright purple beanie and a neo...,{},tradeoff demo,checkable-details,2
3,EXP_C_EXTREME_TRADEOFF_COLORS,d43f647f484e,baseline,increasing,1.0,2.0,40,1337,P_HAT_JACKET_COLORS,A man wearing a bright purple beanie and a neo...,{},tradeoff demo,checkable-details,3
4,EXP_C_EXTREME_TRADEOFF_COLORS,44436a01878e,baseline,increasing,1.0,2.0,40,42,P_CAR_HYDRANT_COLORS,A green sports car parked next to a red fire h...,{},tradeoff demo,checkable-details,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,EXP_C_EXTREME_TRADEOFF_COLORS,3b7e7ce80bec,triangular,decreasing,7.0,100.0,40,1337,P_MUG_BOOK_COLORS,A solid yellow coffee mug resting on a bright ...,{},tradeoff demo,checkable-details,3
212,EXP_C_EXTREME_TRADEOFF_COLORS,a65cdbc383e5,triangular,decreasing,7.0,100.0,40,42,P_HAT_JACKET_COLORS,A man wearing a bright purple beanie and a neo...,{},tradeoff demo,checkable-details,0
213,EXP_C_EXTREME_TRADEOFF_COLORS,308d05784d4e,triangular,decreasing,7.0,100.0,40,1337,P_HAT_JACKET_COLORS,A man wearing a bright purple beanie and a neo...,{},tradeoff demo,checkable-details,1
214,EXP_C_EXTREME_TRADEOFF_COLORS,2b86301213cc,triangular,decreasing,7.0,100.0,40,42,P_CAR_HYDRANT_COLORS,A green sports car parked next to a red fire h...,{},tradeoff demo,checkable-details,2


## Example D — “Out-of-distribution” steering hypothesis (increasing guidance)

Hypothesis:
- For prompts that are *hard / out-of-distribution*, starting with lower guidance may keep samples on-manifold early,
  then increasing guidance later might steer fine details toward adherence.

We encode that by focusing on **increasing** schedules and optionally stronger ceilings.


In [4]:
EXPERIMENT = "EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL"

kinds = ["baseline", "average_constant", "constant", "linear", "exponential", "triangular"]
directions = ["increasing", "decreasing"]
weight_ranges = [
    (1.0, 2.0),   # very weak guidance band
    (1.0, 12.0),  # very strong ceiling (often causes collapse/artifacts)
    (7,50),
    (7,100),
]
seeds = [42, 1337]
num_steps = [40]

prompts = {
    "P_CHAIR_ON_TABLE":  "A small wooden chair placed directly on top of a dining table.",
    "P_COUCH_ON_CEILING" : "A large, heavy couch suspended completely upside down from a living room ceiling.",
    "P_CAR_ON_ROOF" : "A modern sports car parked perfectly balancing on the pitched roof of a suburban house."

}

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,bucket,worker_id
0,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,1645b5d8317e,baseline,increasing,1.0,2.0,40,42,P_CHAIR_ON_TABLE,A small wooden chair placed directly on top of...,{},tradeoff demo,checkable-details,0
1,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,9c2d29981879,baseline,increasing,1.0,2.0,40,1337,P_CHAIR_ON_TABLE,A small wooden chair placed directly on top of...,{},tradeoff demo,checkable-details,1
2,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,260e564b73e4,baseline,increasing,1.0,2.0,40,42,P_COUCH_ON_CEILING,"A large, heavy couch suspended completely upsi...",{},tradeoff demo,checkable-details,2
3,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,25fff2f912d1,baseline,increasing,1.0,2.0,40,1337,P_COUCH_ON_CEILING,"A large, heavy couch suspended completely upsi...",{},tradeoff demo,checkable-details,3
4,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,49b1bc76e59f,baseline,increasing,1.0,2.0,40,42,P_CAR_ON_ROOF,A modern sports car parked perfectly balancing...,{},tradeoff demo,checkable-details,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,5d3213c10a23,triangular,decreasing,7.0,100.0,40,1337,P_CHAIR_ON_TABLE,A small wooden chair placed directly on top of...,{},tradeoff demo,checkable-details,3
212,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,bf200981a752,triangular,decreasing,7.0,100.0,40,42,P_COUCH_ON_CEILING,"A large, heavy couch suspended completely upsi...",{},tradeoff demo,checkable-details,0
213,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,c7c8c9399cef,triangular,decreasing,7.0,100.0,40,1337,P_COUCH_ON_CEILING,"A large, heavy couch suspended completely upsi...",{},tradeoff demo,checkable-details,1
214,EXP_D_EXTREME_TRADEOFF_OOD_SPATIAL,c0a3e5cd2293,triangular,decreasing,7.0,100.0,40,42,P_CAR_ON_ROOF,A modern sports car parked perfectly balancing...,{},tradeoff demo,checkable-details,2


In [ ]:
EXPERIMENT = "EXP_B_EXTREME_TRADEOFF_MATERIAL_TEXTURE_SWAP"

kinds = ["baseline", "average_constant", "constant", "linear", "exponential", "triangular"]
directions = ["increasing", "decreasing"]
weight_ranges = [
    (1.0, 2.0),   # very weak guidance band
    (1.0, 12.0),  # very strong ceiling (often causes collapse/artifacts)
    (7,50),
    (7,100),
]
seeds = [42, 1337]
num_steps = [40]

prompts = {
    "P_FLUFFY_TEACUP":  "A fluffy, furry teacup.",
    "P_COUCH_CONCRETE" : "A living room couch made entirely out of solid, rough grey concrete.",
    "P_BICYCLE_PINK_YARN" : "A traditional bicycle where the frame and wheels are knitted out of pink yarn."

}

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


In [11]:
EXPERIMENT = "EXP_B_EXTREME_TRADEOFF"

kinds = ["constant", "linear", "exponential", "triangular"]
directions = ["increasing", "decreasing"]
weight_ranges = [
    (1.0, 2.0),   # very weak guidance band
    (1.0, 12.0),  # very strong ceiling (often causes collapse/artifacts)
    (7,50),
    (7,100),
]
seeds = [42, 1337]
num_steps = [40]

prompts = {
    "P_APPLE_COUNT_DETAILED": "masterpiece, best quality, 3 red apples on a wooden table, studio lighting, sharp focus",
    "P_VASE_COUNT_DETAILED":  "masterpiece, best quality, 5 ceramic vases on a minimalist shelf, distinct shapes, clean shadows"
}

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    metadata={"notes": "tradeoff demo", "bucket": "checkable-details"},
    num_workers=4
)

df_b = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_b


,experiment_group,experiment_id,kind,direction,w_min,w_max,num_steps,seed,prompt_id,prompt_text,params,notes,bucket,worker_id
0,EXP_B_EXTREME_TRADEOFF,162bc693207e,constant,increasing,1.0,2.0,40,42,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,0
1,EXP_B_EXTREME_TRADEOFF,d3378258bcd1,constant,increasing,1.0,2.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,1
2,EXP_B_EXTREME_TRADEOFF,4b9789b4dda0,constant,increasing,1.0,2.0,40,42,P_VASE_COUNT_DETAILED,"masterpiece, best quality, 5 ceramic vases on ...",{},tradeoff demo,checkable-details,2
3,EXP_B_EXTREME_TRADEOFF,3fba8cdefd65,constant,increasing,1.0,2.0,40,1337,P_VASE_COUNT_DETAILED,"masterpiece, best quality, 5 ceramic vases on ...",{},tradeoff demo,checkable-details,3
4,EXP_B_EXTREME_TRADEOFF,64eb4c98180a,constant,increasing,1.0,12.0,40,42,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,EXP_B_EXTREME_TRADEOFF,853ad4a67b17,exponential,decreasing,7.0,50.0,40,1337,P_VASE_COUNT_DETAILED,"masterpiece, best quality, 5 ceramic vases on ...",{},tradeoff demo,checkable-details,3
140,EXP_B_EXTREME_TRADEOFF,9b659c65502e,exponential,decreasing,7.0,100.0,40,42,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,0
141,EXP_B_EXTREME_TRADEOFF,7073f65d93c8,exponential,decreasing,7.0,100.0,40,1337,P_APPLE_COUNT_DETAILED,"masterpiece, best quality, 3 red apples on a w...",{},tradeoff demo,checkable-details,1
142,EXP_B_EXTREME_TRADEOFF,c4827042ad0f,exponential,decreasing,7.0,100.0,40,42,P_VASE_COUNT_DETAILED,"masterpiece, best quality, 5 ceramic vases on ...",{},tradeoff demo,checkable-details,2


In [ ]:
EXPERIMENT = "EXP_C_OOD_STEERING"

kinds = ["linear", "cosine", "exponential"]
directions = ["increasing"]
weight_ranges = [(1.0, 7.5), (1.0, 10.0)]
seeds = [10, 11, 12]
num_steps = [25]

prompts = {
    "P_OOD1": "A medieval oil painting of a modern smartphone made of stained glass, highly detailed",
    "P_OOD2": "A realistic photo of a transparent wooden chair, studio lighting, clean background",
}

# Example: sweep exponential alpha to change curvature (optional)
params_list = [
    {},                 # default params from registry functions
    {"alpha": 2.0},
    {"alpha": 4.0},
]

spec = ExperimentSpec(
    experiment_group=EXPERIMENT,
    kinds=kinds,
    directions=directions,
    weight_ranges=weight_ranges,
    prompts=prompts,
    seeds=seeds,
    num_steps=num_steps,
    params_list=params_list,
    metadata={"notes": "ood steering", "bucket": "ood"},
    num_workers=3
)

df_c = build_and_write(spec, f"{EXPERIMENT}_plan.csv")
df_c.head(), len(df_c)


## Quick sanity checks
- Any duplicate experiment_ids?
- How many runs per prompt?


In [ ]:
def sanity(df):
    return {
        "rows": len(df),
        "unique_ids": df["experiment_id"].nunique(),
        "dupes": int(len(df) - df["experiment_id"].nunique()),
        "runs_per_prompt": df.groupby("prompt_id").size().to_dict(),
        "kinds": sorted(df["kind"].unique().tolist()),
        "directions": sorted(df["direction"].unique().tolist()),
        "ranges": sorted({(float(a), float(b)) for a, b in zip(df["w_min"], df["w_max"])}),
    }

sanity(df_a), sanity(df_b), sanity(df_c)
